# Joint Inspiral / Ringdown Analysis with PyCBC Inference

This notebook provides an overview of how to do a joint inspiral / ringdown analysis with PyCBC Inference using the method outlined in [arXiv:2312.15146](https://arxiv.org/abs/2312.15146). We suggest that you go through the inference tutorial 8 first for a better understanding of what's done with the ringdown. For more details, see the separate tutorial notebooks.

### Make sure the software is set up ###

To do a ringdown analysis in PyCBC we need the [pykerr](https://github.com/cdcapano/pykerr) package installed. We'll also install `dynesty` for doing the sampling. These are both optional dependencies in the PyCBC repository, meaning that we need to install them separately if we are installing released versions of PyCBC.

In [ ]:
import sys
!{sys.executable} -m pip install pykerr "dynesty<3.0" pycbc --no-cache-dir

In [ ]:
# silence lal warnings
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

# imports we'll need
from pycbc import conversions

## Recap

In tutorial 8 we saw how to do a ringdown analysis of some gravitational wave data using the gated Gaussian model, which used gating and in-painting to excise the pre-merger data. Here, we illustrate how to combine that with an inspiral model the excises the post-merger data. Doing a joint inspiral/ringdown analysis in this manner is necessary if one wishes to marginalize over the sky location and arrival time of a signal, as discussed in [arXiv:2312.15146](https://arxiv.org/abs/2312.15146). To accomplish this, we will use the [HierarchicalModel](https://pycbc.org/pycbc/latest/html/pycbc.inference.models.html#pycbc.inference.models.hierarchical.HierarchicalModel) in PyCBC.

## Injection

To illustrate this, we'll create and analyze a non-spinning GW150914-like signal. We'll inject this into simulated Gaussian noise with a PSD that's similar to the LIGO detectors at the time that GW150914 occurred.

In [ ]:
# the injection parameters
mass1 = 37
mass2 = 32
distance = 400
f_lower = 18
f_ref = 20
approximant = 'IMRPhenomTPHM'

# we'll also get the final mass and spin
mfinal = conversions.final_mass_from_initial(mass1, mass2)
sfinal = conversions.final_spin_from_initial(mass1, mass2)

In [ ]:
injection_config = f"""
[variable_params]

[static_params]
tc = 1126259462.420
mass1 = {mass1}
mass2 = {mass2}
ra = 2.2
dec = -1.25
inclination = 2.5
coa_phase = 0
polarization = 0
distance = {distance}
f_ref = {f_ref}
f_lower = {f_lower}
approximant = {approximant}
taper = start
"""

In [ ]:
!echo '{injection_config}' > injection.ini
!cat injection.ini

In [ ]:
!pycbc_create_injections --verbose \
        --config-files injection.ini \
        --ninjections 1 \
        --seed 10 \
        --output-file injection.hdf \
        --variable-params-section variable_params \
        --static-params-section static_params \
        --dist-section prior \
        --force

## Inference setup: The hierarchical model

In order to do the inspiral and ringdown analysis simulatneously we will use the [HierarchicalModel](https://pycbc.org/pycbc/latest/html/pycbc.inference.models.html#pycbc.inference.models.hierarchical.HierarchicalModel). This is a model of models, which are referred to as "submodels". The hierarchical model calls each sub-models loglikelihood function to get the loglikelihood. The total loglikelihood is the product of likelihoods:

\begin{equation}
\mathcal{L}(\mathbf{d} | \vec{\Theta}; h) = \prod_{k=1}^{n} \mathcal{L}_k(d_k | \vec{\theta}_k, \vec{\theta}_{\rm common}; h)
\end{equation}

where

\begin{equation}
\vec{\Theta} = \{\vec{\theta}_1, ..., \vec{\theta}_n, \vec{\theta}_{\rm common} \}
\end{equation}

are the set of parameters describing all of the sub-models. The set $\vec{\theta}_k$ are the parameters unique to sub-model $k$. The set $\vec{\theta}_{\rm common}$ are shared parameters that are common to all of the sub-models. Note that the hierarhcial model assumes that the likelihoods and data of all of the sub-models are independent of each other, hence, they can be multiplied.

The hierarchical model is a general-use model. Any number of submodels can be provided, and they do not all need to use the same model. This can be used for a number of different analyses (e.g., it can be used to do a joint LISA/ground-based detector analysis as done in [arXiv:2506.01898](https://arxiv.org/abs/2506.01898)). For more on the hierarchical model, see the [example in the PyCBC documentation](https://pycbc.org/pycbc/latest/html/inference/examples/hierarchical.html).

Here, we will use an "inspiral" sub-model for the pre-merger part of the signal and a "ringdown" sub-model for the post-merger. We will use the GatedGaussianNoise model for each region to excise the signal after/before the coalescence time for the inspiral/ringdown sub-model.

To set up the hierarchical model, we to provide a `submodels` argument that specifies the names of the sub-models to use for each analysis. Here, we'll have two sub-models, which we'll name `inspiral` and `ringdown`:

In [ ]:
model_config = """
[model]
name = hierarchical
submodels = inspiral ringdown
"""
!echo '{model_config}' > model.ini

Now we need to provide sections that setup each of the submodels. To do that, we preface `model` with the sub-model name followed by two underscores:

In [ ]:
submodels_config = """
[inspiral__model]
name = gated_gaussian_noise
low-frequency-cutoff = H1:20.0 L1:20.0
ignore-failed-waveforms = 

[ringdown__model]
name = gated_gaussian_noise
low-frequency-cutoff = H1:20.0 L1:20.0
"""

!echo '{submodels_config}' >> model.ini
!cat model.ini

### The data

Since each of our sub-models requires data, we need to provide a data section for each one. As with the model sections, we specify the data for each model by prefixing the relevant `data` section with the sub-model name followed by two underscores. Since both the inspiral and ringdown sub-models use the same data, we can just copy the inspiral data section for the ringdown:

In [ ]:
# these settings are from the BBH example in the pycbc docs, here:
# https://pycbc.org/pycbc/latest/html/inference/examples/bbh.html#simulated-bbh-example
inspiral_data_config = """
[inspiral__data]
instruments = H1 L1
trigger-time = 1126259462.42
analysis-start-time = -6
analysis-end-time = 2
sample-rate = 2048
fake-strain = H1:aLIGOaLIGODesignSensitivityT1800044 L1:aLIGOaLIGODesignSensitivityT1800044
fake-strain-seed = H1:44 L1:45
psd-estimation = median-mean
psd-inverse-length = 8
psd-segment-length = 8
psd-segment-stride = 4
psd-start-time = -256
psd-end-time = 256
channel-name = H1:STRAIN L1:STRAIN
injection-file = injection.hdf
strain-high-pass = 15
pad-data = 8
"""

# we'll just copy the inspiral data to the ringdown
ringdown_data_config = \
    inspiral_data_config.replace('inspiral__', 'ringdown__')

!echo '{inspiral_data_config}' > data.ini
!echo '{ringdown_data_config}' >> data.ini
!cat data.ini

### The prior
Now let's specify the prior. We need to provide static and variable parameters for both sub-models. To specify parameters that are unique to each model, we prefix the parameter name with the sub-model name followed by two underscores. For example, to specify mass1 for the inspiral model, we set `inspiral__mass1`. When the analysis runs, the prefix is stripped from the parameter names by the HierarchicalModel, then passed to each of the sub-models.

To specify parameters that are common to both models, we simply don't put a prefix on the parameter. In this case, we will make the sky location and the coalescence time (which is the switch point from the inspiral to ringdown models; here specified as a `delta_tc`, relative to the common trigger time) common parameters.

For demonstration purposes here, we'll fix everything but the timing and sky location parameters. We will vary the phase for the inspiral and QNM modes; we will also vary the amplitude of the QNM modes. We will also need to indpendently vary the inclination angle for each model. Althought they in-principle can share the same inclination, because it is defined differently for the IMR template and the QNM template, we just vary it independently. In a real analysis, we would vary all of the intrinsic parameters (component masses/spins for the inspiral; final mass and spin for the ringdown).

In [ ]:
# we'll fix most parameters for demonstration purposes
static_params = f"""
[static_params]
trigger_time = 1126259462.43
polarization = 0
# inspiral parameters
inspiral__f_lower = {f_lower}
inspiral__f_ref = {f_ref}
inspiral__approximant = {approximant}
inspiral__mass1 = {mass1}
inspiral__mass2 = {mass2}
inspiral__distance = {distance}
inspiral__taper = start
# ringdown parameters
ringdown__approximant = TdQNMfromFinalMassSpin
ringdown__lmns = 222
ringdown__f_final = 2048
ringdown__final_mass = {mfinal}
ringdown__final_spin = {sfinal}
"""
!echo '{static_params}' > prior.ini

In [ ]:
variable_params = """
[variable_params]
# shared parameters
ra =
dec =
delta_tc =
# inspiral parameters
inspiral__inclination =
inspiral__coa_phase = 
# ringdown parameters
ringdown__inclination =
ringdown__amp220 = 
ringdown__phi220 = 
ringdown__amp221 =
ringdown__phi221 =


[prior-delta_tc]
name = uniform
min-delta_tc = -0.05
max-delta_tc = 0.05

[prior-inspiral__inclination]
name = sin_angle

[prior-inspiral__coa_phase]
name = uniform_angle

[prior-ringdown__inclination]
name = sin_angle

[prior-ringdown__amp220]
name = uniform_log10
min-ringdown__amp220 = 1e-25
max-ringdown__amp220 = 8e-17

[prior-ringdown__phi220]
name = uniform_angle

[prior-ringdown__amp221]
name = uniform
min-ringdown__amp221 = 0
max-ringdown__amp221 = 5

[prior-ringdown__phi221]
name = uniform_angle

[prior-ra+dec]
name = uniform_sky

[waveform_transforms-tc]
name = custom
inputs = trigger_time, delta_tc
tc = trigger_time + delta_tc

[waveform_transforms-inspiral__t_gate_start]
name = custom
inputs = tc
inspiral__t_gate_start = tc

[waveform_transforms-inspiral__t_gate_end]
name = custom
inputs = tc
inspiral__t_gate_end = tc + 1.

[waveform_transforms-ringdown__t_gate_start]
name = custom
inputs = tc
ringdown__t_gate_start = tc - 1.

[waveform_transforms-ringdown__t_gate_end]
name = custom
inputs = tc
ringdown__t_gate_end = tc
"""
!echo '{variable_params}' >> prior.ini
!cat prior.ini

### The sampler

Finally, we need to specify the sampler settings. As with tutorial 8, we'll use dynesty with some small number of parameters to get to a checkpoint quickly.

In [ ]:
sampler_config = """
[sampler]
name = dynesty
dlogz = 1
nlive = 100
checkpoint_time_interval = 30
maxcall = 100
"""
!echo '{sampler_config}' > sampler.ini
!cat sampler.ini

### Run `pycbc_inference`

Now that we've set up our config files, we can run `pycbc_inference`. Even though we've simplified the setup, this would still take a long time to run in a notebook like this. Instead, start this running, then interrupt the kernel after you see the output message say that it has checkpointed a couple times (should occur within a couple minutes).

In [ ]:
# generate a random int for the seed
import numpy
seed = numpy.random.randint(1,int(2**31))
print("Using seed: {}".format(seed))

In [ ]:
!timeout 300 \
pycbc_inference --verbose \
    --config-files model.ini data.ini prior.ini sampler.ini \
    --output-file inference.hdf \
    --seed {seed} \
    --nprocesses 8 \
    --force    

We can see that a checkpoint file exists in our directory:

In [ ]:
!ls -lh inference.hdf.checkpoint

## Plotting with `pycbc_inference_plot_posterior`

We'll just plot the raw samples from the checkpoint. This won't be very informative since we did not run long; it's just to illustrate that we have some points saved:

In [ ]:
!pycbc_inference_plot_posterior --verbose \
    --input-file inference.hdf.checkpoint \
    --output-file raw_samples_current.png \
    --plot-scatter --plot-marginal \
    --z-arg loglikelihood \
    --raw-samples

In [ ]:
from IPython.display import Image
Image('raw_samples_current.png', height=480)